In [2]:
import pandas as pd

In [194]:
municipality = pd.read_csv("clean_data/turkey_municipality.csv")
parlimentary = pd.read_csv("clean_data/turkish_parlimentary.csv")
parlimentary_d = pd.read_csv("clean_data/milletvekili_detailed.csv")
party_alignment = pd.read_excel("clean_data/party_alignment.xlsx")
events = pd.read_excel("data/turkish_catastrophic_event.xlsx")

In [195]:
parlimentary = parlimentary.sort_values(by = ["year", "turnout"], ascending = False)
parlimentary = parlimentary.drop_duplicates(subset = ["year"], keep = "first")

# Municipality

In [107]:
municipality.drop(columns = ['prev_year',
       'winner', "prev_winner", "swing"], inplace = True)
municipality.city = municipality.city.str.replace("mersin", "icel")
municipality = municipality[(municipality.city != "turkiye") & (municipality.city != "altkademebelediyesi")]
municipality["percent_turnout"] = municipality["turnout"] / municipality["registered_voters"]

In [110]:
def determine_winner(row):
    row = row.dropna(axis=0, how='any')
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [111]:
years = list(municipality.year.unique())
years = sorted(years, reverse = True)
municipality["prev_year"] = municipality.year.apply(lambda row: None if row == 1989 else years[years.index(row) + 1])

In [112]:
municipality["winner"] = municipality.apply(lambda row: determine_winner(row), axis = 1)
municipality["winner"] = municipality["winner"].str.split("_", expand = True)[0]

In [115]:
municipality_results = municipality[["city", "year", "winner"]].drop_duplicates()
municipality_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
municipality = municipality.merge(municipality_results, on = ["city", "prev_year"], how = "left")

In [116]:
municipality["swing"] = ((municipality["winner"] != municipality["prev_winner"]) & (municipality["prev_winner"].notna())).astype(int)

In [117]:
municipality.winner = municipality.winner.str.lower()
municipality.prev_winner = municipality.prev_winner.str.lower()
municipality.winner = municipality.winner.str.replace("ak parti", "akp")
municipality.prev_winner = municipality.prev_winner.str.replace("ak parti", "akp")

In [118]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
party_alignment.winner = party_alignment.winner.str.replace("ak parti", "akp")
municipality = municipality.merge(party_alignment, on = "winner", how = "left")

In [119]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]

In [121]:
municipality = municipality.merge(party_alignment, on = "prev_winner", how = "left")
municipality.to_csv("clean_data/turkey_municipality.csv", index = False)

# Parlimentary

In [196]:
parlimentary.drop(columns = ["num_milletvekili", "toplam_gecerli_oy_sayisi", "milletvekili_sayisi", "bgmztotal_votes"], inplace = True)
parlimentary.rename(columns = {"registered_voter": "registered_voters",
                             'valid_votes': "valid_votes"}, inplace = True)
cols = [i for i in parlimentary.columns if "percent_votes" not in i]
parlimentary = parlimentary[cols]
parlimentary.columns = [i.replace("total_votes", "") for i in parlimentary.columns]
parlimentary["city"] = ["turkiye"]*len(parlimentary)

In [197]:
years = list(parlimentary.year.unique())
years = sorted(years, reverse = True)
parlimentary["prev_year"] = parlimentary.year.apply(lambda row: None if row == "1950" else years[years.index(row) + 1])

In [198]:
def determine_winner_p(row):
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.dropna(axis=0, how='any')
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [199]:
parlimentary["winner"] = parlimentary.apply(lambda row: determine_winner_p(row), axis = 1)
parlimentary["winner"] = parlimentary["winner"].str.split("_", expand = True)[0]
parlimentary.winner = parlimentary.winner.str.replace("ak", "akp")

In [200]:
parlimentary_results = parlimentary[["year", "winner"]].drop_duplicates()
parlimentary_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
parlimentary = parlimentary.merge(parlimentary_results, on = ["prev_year"], how = "left")
parlimentary.loc[18, "prev_winner"] = "chp"

In [201]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
parlimentary = parlimentary.merge(party_alignment, on = "winner", how = "left")

In [202]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]
parlimentary = parlimentary.merge(party_alignment, on = "prev_winner", how = "left")

In [210]:
parlimentary["swing"] = ((parlimentary["winner"] != parlimentary["prev_winner"]) & (parlimentary["prev_winner"].notna())).astype(int)

In [211]:
parlimentary.winner = parlimentary.winner.str.lower()
parlimentary.prev_winner = parlimentary.prev_winner.str.lower()
parlimentary.winner = parlimentary.winner.str.replace("ak parti", "akp")
parlimentary.prev_winner = parlimentary.prev_winner.str.replace("ak parti", "akp")

# Concat Municipality and Parlimentary

In [212]:
elections_all = pd.concat([parlimentary, municipality], ignore_index = True)

In [219]:
elections_all.to_csv("clean_data/elections_all.csv", index = False)

# Events

In [11]:
events = pd.get_dummies(events)
events.to_csv("clean_data/turkish_crises.csv", index = False)

In [221]:
elections_all.year.unique()

array(['2018', '2015_2', '2015_1', '2011', '2007', '2002', '1999', '1995',
       '1991', '1987', '1983', '1977', '1973', '1969', '1965', '1961',
       '1957', '1954', '1950', 2019, 2009, 1994, 2004, 2014, 1999, 1989],
      dtype=object)

In [226]:
events.year.unique()

array([1960, 1970, 1971, 1974, 1979, 1980, 1984, 1994, 1997, 2001, 2007,
       2013, 2015, 2016, 2018])

In [223]:
elections_all = elections_all.merge(events, on = "year", how = "left")

In [225]:
elections_all[elections_all["crisis type"].notna()]

,year,registered_voters,turnout,percent_turnout,valid_vote,ak_parti,anadolu_partisi,anap,atp,bbp,...,dyp_percent,IDP_vote,idp_percent,MÇP_vote,mcp_percent,RP_vote,rp_percent,SHP_vote,shp_percent,crisis type
65,1994,10904360.0,9739574.0,0.893182,9143354.0,NaN,NaN,1992281.0,NaN,35057.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
66,1994,1670867.0,1525392.0,0.912934,1439838.0,NaN,NaN,268519.0,NaN,8649.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
67,1994,260054.0,230050.0,0.884624,214775.0,NaN,NaN,40895.0,NaN,641.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
68,1994,590162.0,554609.0,0.939757,518917.0,NaN,NaN,144309.0,NaN,1409.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
69,1994,169466.0,119754.0,0.706655,86302.0,NaN,NaN,18826.0,NaN,672.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
70,1994,133846.0,119082.0,0.889694,111942.0,NaN,NaN,23775.0,NaN,1978.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
71,1994,282758.0,268413.0,0.949268,256919.0,NaN,NaN,49045.0,NaN,546.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
72,1994,325128.0,279437.0,0.859468,255113.0,NaN,NaN,37042.0,NaN,1731.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
73,1994,239391.0,193867.0,0.809834,176698.0,NaN,NaN,65879.0,NaN,481.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
74,1994,4585817.0,4078906.0,0.889461,3865126.0,NaN,NaN,855897.0,NaN,13662.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,economic
